<a href="https://colab.research.google.com/github/biomedist/OssData/blob/main/midtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오픈소스데이터 중간과제물


In [55]:
import requests
import pandas as pd
import json


# 2015년 1월부터 2024년 12월까지의 전국 평균 에너지 사용금액 데이터 수집
def get_year_months():
    year_months = []
    for year in range(2015, 2024+1):
        for month in range(1,12+1):
            year_months.append(f'{year}{month:02d}')
    return year_months

# URL,API정보
url = "http://apis.data.go.kr/1613000/ApHusEnergyUseInfoOfferServiceV2/getWntyAvrgEnergyUseAmountInfoSearchV2"
api_key = '6f8923bf4930eaf1f5dc16e18bd2505221d140710cd395084c84d15bef5791b0'


# 응답데이터 저장
responses = []

# 연월 리스트통해 데이터 수집
for year_month in get_year_months():
    params = {
            'serviceKey': api_key,
            'searchDate': year_month,
            'type': 'json'
    }

# 데이터 수집
    response = requests.get(url, params=params)
    data = response.json()['response']['body']['item']
    data['year_month'] = year_month

    responses.append(data)

# 호출 성공/실패 출력
print(response.json())





{'response': {'body': {'item': {'heat': 515, 'waterHot': 155, 'gas': 3, 'elect': 711, 'waterCool': 241}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}


In [2]:
response.json()

{'response': {'body': {'item': {'heat': 0,
    'waterHot': 0,
    'gas': 0,
    'elect': 0,
    'waterCool': 0}},
  'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}

In [73]:
# pandas DataFrame으로 변환
df = pd.DataFrame(responses)

# JSON형식으로 저장
df.to_json('energy_use.json', indent=4, force_ascii=False)

print(df)
print(df.index)
print(df.columns)

print(df.info())

## 데이터기본정보 출력
# 데이터측정
df.describe()

     heat  waterHot  gas  elect  waterCool year_month
0     532       132    5    674        176     201501
1     435       126    5    639        174     201502
2     329       123    4    532        173     201503
3     157       106    3    554        180     201504
4      51        88    2    506        184     201505
..    ...       ...  ...    ...        ...        ...
115    28        51    1   1292        258     202408
116    28        53    1   1081        253     202409
117    64        91    1    696        247     202410
118   210       118    2    646        243     202411
119   515       155    3    711        241     202412

[120 rows x 6 columns]
RangeIndex(start=0, stop=120, step=1)
Index(['heat', 'waterHot', 'gas', 'elect', 'waterCool', 'year_month'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   heat        12

,heat,waterHot,gas,elect,waterCool
count,120.000000,120.000000,120.000000,120.000000,120.000000
mean,168.800000,86.550000,1.975000,595.758333,221.208333
std,160.219471,32.606606,1.016358,142.964178,19.217418
min,21.000000,27.000000,1.000000,399.000000,173.000000
25%,28.000000,54.000000,1.000000,493.750000,208.500000
50%,94.500000,89.000000,2.000000,555.500000,225.000000
75%,305.250000,112.250000,3.000000,656.500000,235.000000
max,544.000000,159.000000,5.000000,1292.000000,258.000000


In [15]:
## JSON 형식 데이터 저장
df.to_json('energy_use.json', indent=4, force_ascii=False)  #indent넣기, 한글을 위해 아스키넣기

# 데이터 측정
df.describe()

# 모든 데이터 타입 측정
df.describe(include='all')

# 수치형 데이터 측정
df.describe(include=[np.number])



,heat,waterHot,gas,elect,waterCool
count,1.0,1.0,1.0,1.0,1.0
mean,0.0,0.0,0.0,0.0,0.0
std,NaN,NaN,NaN,NaN,NaN
min,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0


ValueError: If using all scalar values, you must pass an index